<a href="https://colab.research.google.com/github/Taweilo/rag_gpt4.0/blob/main/RAG_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install openai
!pip install tiktoken
!pip install faiss-cpu
!pip install langchain_experimental
!pip install "langchain[docarray]"
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 114.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successf

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.indexes import VectorstoreIndexCreator
from langchain_experimental.agents.agent_toolkits.csv.base import create_csv_agent
from langchain.agents.agent_types import AgentType
from langchain.memory import ConversationBufferMemory
import tiktoken

In [ ]:
import os
from google.colab import userdata

# Prompt the user for their OpenAI API key
api_key = userdata.get('openai')

# Set the API key as an environment variable
os.environ["OPENAI_API_KEY"] = api_key

# Optionally, check that the environment variable was set correctly
print("OPENAI_API_KEY has been set!")


OPENAI_API_KEY has been set!


In [ ]:
from google.colab import files

# Upload file manually
uploaded = files.upload()

# Get the uploaded file name
txt_file_path = list(uploaded.keys())[0]

# Read the file content
with open(txt_file_path, 'r', encoding='utf-8') as file:
    data = file.read()

# Display the first few lines
print(data[:500])  # Adjust as needed

Saving scalexi.txt to scalexi (2).txt
ScaleX Innovation Bio

ScaleX Innovation is a pioneering leader in the realm of Generative AI and Large Language Models. With a keen focus on integrating these transformative technologies into business strategies, the company has carved a niche for itself by offering tailored solutions that enhance innovation and operational efficiency. ScaleX's proficiency extends across multiple industry verticals, ensuring that businesses can harness the power of AI-driven digital transformation. Whether it's


In [ ]:
from langchain.schema import Document
from langchain.text_splitter import CharacterTextSplitter
# Convert `data` into a list of Document objects
data = [Document(page_content=data)]

# Initialize text splitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# Split data
split_data = text_splitter.split_documents(data)

# Display the number of chunks
print(f"Number of chunks: {len(split_data)}")
print(split_data[0].page_content)  # Display the first chunk

Number of chunks: 6
ScaleX Innovation Bio

ScaleX Innovation is a pioneering leader in the realm of Generative AI and Large Language Models. With a keen focus on integrating these transformative technologies into business strategies, the company has carved a niche for itself by offering tailored solutions that enhance innovation and operational efficiency. ScaleX's proficiency extends across multiple industry verticals, ensuring that businesses can harness the power of AI-driven digital transformation. Whether it's automating workflows, content analysis, or custom model implementations, ScaleX Innovation stands at the cutting-edge, committed to bridging the gap between technology and business. Their dedication to ethical compliance and versatility makes them a trusted partner for businesses worldwide.س


In [ ]:
# Create vector store
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(data, embedding=embeddings)

In [ ]:
# Create conversation chain
llm = ChatOpenAI(temperature=0.3, model_name="gpt-4")
memory = ConversationBufferMemory(
memory_key='chat_history', return_messages=True)
conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(),
        memory=memory
        )

<ipython-input-34-03e2f865a848>:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0.7, model_name="gpt-4")
<ipython-input-34-03e2f865a848>:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [ ]:
query = "What is ScaleX Innovation?"
result = conversation_chain({"question": query})
answer = result["answer"]
answer

<ipython-input-35-cd5c62f6905b>:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = conversation_chain({"question": query})


'ScaleX Innovation is a leading company in the field of Generative AI and Large Language Models. The company focuses on integrating these technologies into business strategies to enhance innovation and operational efficiency. They offer tailored solutions that include automating workflows, content analysis, and custom model implementations. ScaleX Innovation operates across multiple industry verticals, enabling businesses to utilize AI-driven digital transformation. They are committed to ethical compliance and offer versatile solutions, making them a trusted partner for businesses globally.'

In [ ]:
query = "What is the contact information?"
result = conversation_chain({"question": query})
answer = result["answer"]
answer

'The contact information for ScaleX Innovation is as follows:\n\nAddress: Pavillon d’Or Bldg, Route Mahdia Km 0.5, 3000 Sfax, Tunisia.\nEmail Address: info@scalexi.ai\nPhone Number: ++216-55-770-606\nFax: +216-55-770-606'

In [ ]:
query = "What are the main activities of ScaleX Innovation. Write is as three bullet points."
result = conversation_chain({"question": query})
answer = result["answer"]
answer

'- ScaleX Innovation specializes in Generative AI and Large Language Models, providing bespoke solutions for businesses to drive innovation and automation.\n- They offer AI-enabled content and data analysis, customizing Large Language Models for specific business strategies.\n- ScaleX Innovation also provides cross-domain consultation for business automation, ensuring that their AI solutions comply with industry regulations and ethical standards.'

# Gradio

# RAG on colab
1. Upload a PDF.
2. Process the PDF: Store, chunk, embed, and save embeddings in a pickle file.
3. Query the data: Transform the query into embeddings, find similar chunks, and print relevant text.

In [ ]:
! pip install gradio
! pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.7/298.7 kB 10.5 MB/s eta 0:00:00


In [ ]:
import gradio as gr
import os
import openai
from langchain.document_loaders import PyPDFLoader
from langchain.schema import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from google.colab import userdata

# Prompt the user for their OpenAI API key
api_key = userdata.get('openai')

# Set the API key as an environment variable
os.environ["OPENAI_API_KEY"] = api_key


# Initialize global variables
vectorstore = None
conversation_chain = None

# Function to process the uploaded PDF
def process_pdf(pdf_file):
    global vectorstore, conversation_chain

    # Load and extract text from the PDF
    loader = PyPDFLoader(pdf_file.name)
    documents = loader.load()

    # Split text into chunks
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    split_data = text_splitter.split_documents(documents)

    # Create vector store
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(split_data, embedding=embeddings)

    # Create conversation chain
    llm = ChatOpenAI(temperature=0.3, model_name="gpt-4")
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever(), memory=memory
    )

    return "PDF processed! You can now ask questions."

# Function to handle user queries
def chatbot(query, chat_history):
    global conversation_chain

    if conversation_chain is None:
        return "Please upload a PDF first.", chat_history

    # Get answer from LLM
    result = conversation_chain({"question": query})
    answer = result["answer"]

    # Update chat history
    chat_history.append((query, answer))

    return answer

# Create Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("## 📄 PDF-Based RAG Chatbot with GPT-4")

    # PDF Upload Section
    with gr.Row():
        pdf_input = gr.File(label="Upload PDF", type="filepath")
        process_button = gr.Button("Process PDF")

    status_output = gr.Textbox(label="Status", interactive=False)

    # Chat Interface
    chatbot_interface = gr.ChatInterface(fn=chatbot)

    # Define button functionality
    process_button.click(process_pdf, inputs=[pdf_input], outputs=[status_output])

# Launch the Gradio App
demo.launch(debug=True)

/usr/local/lib/python3.11/dist-packages/gradio/components/chatbot.py:288: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://60dad1c5b1ce6bbf87.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://60dad1c5b1ce6bbf87.gradio.live


In [ ]:
import gradio as gr
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain.chat_models import ChatOpenAI
import os
from google.colab import userdata

# Prompt the user for their OpenAI API key
api_key = userdata.get('openai')

# Set the API key as an environment variable
os.environ["OPENAI_API_KEY"] = api_key

# Global variables
vectorstore = None
conversation_chain = None

# Function to process the uploaded PDF
def process_pdf(pdf_file):
    global vectorstore, conversation_chain

    if pdf_file is None:
        return "No file uploaded. Please upload a PDF."

    # Load and extract text from the PDF
    loader = PyPDFLoader(pdf_file.name)
    documents = loader.load()

    # Split text into chunks
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    split_data = text_splitter.split_documents(documents)

    # Create vector store
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(split_data, embedding=embeddings)

    # Create conversation chain
    llm = ChatOpenAI(temperature=0.3, model_name="gpt-4")
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever(), memory=memory
    )

    return "PDF processed! You can now ask questions."

# Function to handle user queries
def chatbot(query, chat_history):
    global conversation_chain

    if conversation_chain is None:
        return "Please upload and process a PDF first.", chat_history

    # Get answer from LLM
    result = conversation_chain({"question": query})
    answer = result["answer"]

    # Update chat history
    chat_history.append({"role": "user", "content": query})
    chat_history.append({"role": "assistant", "content": answer})

    return chat_history

# Create Gradio Interface
with gr.Blocks() as demo:
    gr.Markdown("## 📄 PDF-Based RAG Chatbot with GPT-4.0")

    # PDF Upload Section
    with gr.Row():
        pdf_input = gr.File(label="Upload PDF", type="filepath")
        process_button = gr.Button("Process PDF")

    status_output = gr.Textbox(label="Status", interactive=False)

    # Chat Interface
    chatbot_interface = gr.Chatbot(label="Chat with your document", type="messages")
    query_input = gr.Textbox(label="Your Question")
    submit_button = gr.Button("Ask")

    # Example Queries
    example_queries = gr.Examples(
        examples=[
            ["Summarize the document"],
            ["List the most common questions about the document"]
        ],
        inputs=[query_input],
    )

    # Define button functionality
    process_button.click(process_pdf, inputs=[pdf_input], outputs=[status_output])
    submit_button.click(chatbot, inputs=[query_input, chatbot_interface], outputs=[chatbot_interface])

# Launch the Gradio App
demo.launch(debug=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://5b0cfb5ba2ccca762f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://5b0cfb5ba2ccca762f.gradio.live


Reference:
- https://hackmd.io/@flagmaker/BJRSsgY1C
- https://scalexi.medium.com/implementing-a-retrieval-augmented-generation-rag-system-with-openais-api-using-langchain-ab39b60b4d9f